In [1]:
pip install --upgrade numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 55.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.2 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.2 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.2 which 

In [1]:
pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 108.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-ozd0852l/gym-walk_3e46b88da18a4fea9f2a93ebc1a6ba8e
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-ozd0852l/gym-walk_3e46b88da18a4fea9f2a93ebc1a6ba8e
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=65c9a557d33ce5eb0a0e1d49ee5b614e77d9eb4cb25e493782ed39c7c4209ed2
  Stored in directory: /tmp/pip-ephem-wheel-cache-1zzz1r0i/wheels/bf/23/e5/a94be4a90dd18f7ce958c21f192276cb01ef0daaf2bc66583b
Successfully built gym-walk


In [2]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [70]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k: action_symbols[k % len(action_symbols)] for k in range(len(P[0]))}

    for s in range(len(P)):
        # Handle both array and function policies
        a = pi[s] if isinstance(pi, np.ndarray) else pi(s)

        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0:
            print("|")


In [71]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [72]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.reset(seed=123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            # Handle both array and function policy
            action = pi[state] if isinstance(pi, (list, np.ndarray)) else pi(state)
            state, _, done, h = env.step(action)
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [73]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.reset(seed=123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            # ✅ handle both array/list and function policy
            action = pi[state] if isinstance(pi, (list, np.ndarray)) else pi(state)
            state, reward, done, _ = env.step(action)
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [74]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
LEFT, DOWN, RIGHT, UP = range(4)

In [75]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [76]:
init_state

0

In [77]:
state, reward, done, info = env.step(RIGHT)
print("state:{0} - reward:{1} - done:{2} - info:{3}".format(state, reward, done, info))

state:1 - reward:0.0 - done:False - info:{'prob': 0.3333333333333333}


In [114]:
# Adversarial Policy
pi_frozenlake1 = lambda s: {
    0: RIGHT,
    1: UP,
    2: LEFT,
    3: RIGHT,
    4: DOWN,
    5: RIGHT,
    6: RIGHT,
    7: RIGHT,
    8: LEFT,
    9: RIGHT,
    10:RIGHT,
    11:UP,
    12:RIGHT,
    13:DOWN,
    14:RIGHT,
    15:RIGHT #Stop
}[s]
print("Name: Shehan Shajahan")
print("Register Number: 212223240154")
print_policy(pi_frozenlake1, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Name: Shehan Shajahan
Register Number: 212223240154
Policy:
| 00      > | 01      ^ | 02      < | 03      > |
| 04      v |           | 06      > |           |
| 08      < | 09      > | 10      > |           |
|           | 13      v | 14      > |           |


In [115]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_frozenlake1, goal_state=goal_state)*100,
    mean_return(env, pi_frozenlake1)))

Reaches goal 3.00%. Obtains an average undiscounted return of 0.0300.


In [116]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)
    while True:
        prev_V = V.copy()
        for s in range(len(P)):
            # ✅ handle both array/list and function policy
            action = pi[s] if isinstance(pi, (list, np.ndarray)) else pi(s)
            V[s] = sum(
                prob * (reward + gamma * prev_V[next_state] * (not done))
                for prob, next_state, reward, done in P[s][action]
            )
        # convergence check
        if np.max(np.abs(prev_V - V)) < theta:
            break
    return V

In [117]:
# Code to evaluate the adversarial policy
V1 = policy_evaluation(pi_frozenlake1, P)
print("Name: Shehan Shajahan")
print("Register Number: 212223240154")
print_state_value_function(V1, P, n_cols=4, prec=5)

Name: Shehan Shajahan
Register Number: 212223240154
State-value function:
| 00 0.02703 | 01 0.05405 | 02 0.08108 | 03    0.0 |
| 04    0.0 |           | 06 0.10811 |           |
| 08    0.0 | 09 0.18468 | 10 0.24324 |           |
|           | 13 0.31081 | 14 0.62162 |           |


In [118]:
def policy_improvement(V, P, gamma=1.0):
    Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
    for s in range(len(P)):
        for a in range(len(P[s])):
            for prob, next_state, reward, done in P[s][a]:
                Q[s, a] += prob * (reward + gamma * V[next_state] * (not done))
    new_pi = np.argmax(Q, axis=1)
    return new_pi

In [119]:
pi_2 = policy_improvement(V1, P)
print("Name: Shehan Shajahan")
print("Register Number: 212223240154")
print_policy(pi_2, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Name: Shehan Shajahan
Register Number: 212223240154
Policy:
| 00      ^ | 01      ^ | 02      < | 03      < |
| 04      < |           | 06      < |           |
| 08      v | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [120]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_2, goal_state=goal_state)*100,
    mean_return(env, pi_2)))

Reaches goal 25.00%. Obtains an average undiscounted return of 0.2500.


In [121]:
V2 = policy_evaluation(pi_2, P)
print("Name: Shehan Shajahan")
print("Register Number: 212223240154")
print_state_value_function(V2, P, n_cols=4, prec=5)

Name: Shehan Shajahan
Register Number: 212223240154
State-value function:
| 00 0.25641 | 01 0.25641 | 02 0.25641 | 03 0.12821 |
| 04 0.24359 |           | 06 0.25641 |           |
| 08 0.23077 | 09 0.46154 | 10 0.51282 |           |
|           | 13 0.64103 | 14 0.82051 |           |


In [122]:
# comparing the initial and the improved policy
if(np.sum(V1>=V2)==16):
  print("The Adversarial policy is the better policy")
elif(np.sum(V2>=V1)==16):
  print("The Improved policy is the better policy")
else:
  print("Both policies have their merits.")

The Improved policy is the better policy


In [123]:
def policy_iteration(P, gamma=1.0, theta=1e-10):
    # 1. Initialize policy (random)
    pi = np.zeros(len(P), dtype=int)

    while True:
        # 2. Policy Evaluation
        V = policy_evaluation(pi, P, gamma=gamma, theta=theta)

        # 3. Policy Improvement
        Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
        for s in range(len(P)):
            for a in range(len(P[s])):
                Q[s, a] = sum(
                    prob * (reward + gamma * V[next_state] * (not done))
                    for prob, next_state, reward, done in P[s][a]
                )
        new_pi = np.argmax(Q, axis=1)

        # 4. Check if policy has stabilized
        if np.array_equal(pi, new_pi):
            break
        pi = new_pi

    return V, pi

In [124]:
optimal_V, optimal_pi = policy_iteration(P)

In [125]:
print("Name: Shehan Shajahan")
print("Register Number: 212223240154")
print('Optimal policy and state-value function (PI):')
print_policy(optimal_pi, P, action_symbols=('<', '>'), n_cols=7)

Name: Shehan Shajahan
Register Number: 212223240154
Optimal policy and state-value function (PI):
Policy:
| 00      < | 01      > | 02      > | 03      > | 04      < |           | 06      < |
|           | 08      > | 09      > | 10      < |           |           | 13      < |
| 14      > |           

In [126]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, optimal_pi, goal_state=goal_state)*100,
    mean_return(env, optimal_pi)))

Reaches goal 69.00%. Obtains an average undiscounted return of 0.6900.


In [127]:
print("Name: Shehan Shajahan")
print("Register Number: 212223240154")
print_state_value_function(optimal_V, P, n_cols=7, prec=5)

Name: Shehan Shajahan
Register Number: 212223240154
State-value function:
| 00 0.82353 | 01 0.82353 | 02 0.82353 | 03 0.82353 | 04 0.82353 |           | 06 0.52941 |
|           | 08 0.82353 | 09 0.82353 | 10 0.76471 |           |           | 13 0.88235 |
| 14 0.94118 |           